In [ ]:
# Import our dependencies module 19.4.3
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

In [ ]:
# Import our dependencies module 19.4.3
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

In [ ]:
# Import our input dataset
worksheet = gc.open('fl_test_covid_df').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
print(rows)

import pandas as pd
df = pd.DataFrame.from_records(rows)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [20]:
# Import our input dataset
url="https://raw.githubusercontent.com/dlagorealestate/Exploritory-CDC-Data/main/fl_test_covid_df.csv"
df=pd.read_csv(url)
df


,Unnamed: 0,age_group,death_yn
0,0,0 - 17 years,No
1,1,18 to 49 years,No
2,2,0 - 17 years,No
3,3,18 to 49 years,No
4,4,0 - 17 years,No
...,...,...,...
869395,869395,18 to 49 years,No
869396,869396,18 to 49 years,No
869397,869397,18 to 49 years,No
869398,869398,18 to 49 years,No


In [21]:
# Generate our categorical variable list
attrition_cat = df.dtypes[df.dtypes == "object"].index.tolist()
attrition_cat

['age_group', 'death_yn']

In [22]:
# Check the number of unique values in each column
df[attrition_cat].nunique()

age_group    4
death_yn     2
dtype: int64

In [23]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(df[attrition_cat]))

# Add the encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names(attrition_cat)
encode_df.head()

,age_group_0 - 17 years,age_group_18 to 49 years,age_group_50 to 64 years,age_group_65+ years,death_yn_No,death_yn_Yes
0,1.0,0.0,0.0,0.0,1.0,0.0
1,0.0,1.0,0.0,0.0,1.0,0.0
2,1.0,0.0,0.0,0.0,1.0,0.0
3,0.0,1.0,0.0,0.0,1.0,0.0
4,1.0,0.0,0.0,0.0,1.0,0.0


In [25]:
# Split our preprocessed data into our features and y target arrays want to predict
y = encode_df["death_yn_Yes"].values
X = encode_df.drop(["death_yn_Yes","death_yn_No"],1).values

# Split the preprocessed data into a training and testing dataset useing sklearn
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)


In [26]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [27]:
# Create the Keras Sequential model
nn_model = tf.keras.models.Sequential()

# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  4


# Add our first Dense layer, including the input layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, activation="relu", input_dim=number_input_features))

# Add the output layer that uses a probability activation function
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))




In [28]:
# Check the structure of the Sequential model
nn_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 4)                 20        
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 5         
Total params: 25
Trainable params: 25
Non-trainable params: 0
_________________________________________________________________


In [29]:
# Compile the Sequential model together and customize metrics
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [30]:
# Fit the model to the training data
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
20377/20377 [==============================] - 25s 1ms/step - loss: 0.1216 - accuracy: 0.9638
Epoch 2/50
20377/20377 [==============================] - 24s 1ms/step - loss: 0.1010 - accuracy: 0.9703
Epoch 3/50
20377/20377 [==============================] - 24s 1ms/step - loss: 0.1010 - accuracy: 0.9703
Epoch 4/50
20377/20377 [==============================] - 25s 1ms/step - loss: 0.1010 - accuracy: 0.9703
Epoch 5/50
20377/20377 [==============================] - 24s 1ms/step - loss: 0.1010 - accuracy: 0.9703
Epoch 6/50
20377/20377 [==============================] - 24s 1ms/step - loss: 0.1010 - accuracy: 0.9703
Epoch 7/50
20377/20377 [==============================] - 24s 1ms/step - loss: 0.1010 - accuracy: 0.9703
Epoch 8/50
20377/20377 [==============================] - 24s 1ms/step - loss: 0.1010 - accuracy: 0.9703
Epoch 9/50
20377/20377 [==============================] - 25s 1ms/step - loss: 0.1010 - accuracy: 0.9703
Epoch 10/50
20377/20377 [==============================

In [31]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

6793/6793 - 5s - loss: 0.1009 - accuracy: 0.9702
Loss: 0.10094590485095978, Accuracy: 0.970232367515564
